In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

Initial feed and marking landmarks

In [3]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False


        results=holistic.process(image)


        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(0,0,0),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))


        cv2.imshow("feed",image)
        
        if cv2.waitKey(100) & 0xFF==ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

In [26]:
results.face_landmarks.landmark[0].visibility

0.0

CSV

In [3]:
import os
import csv

In [5]:
coord=len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)+21+21
coord

543

In [6]:
landmarks=["class","subclass","condition"]
for i in range(1,coord+1):
    landmarks+=["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]


In [7]:
with open("coordinates.csv",mode="w",newline='') as f:
    csv_writer=csv.writer(f,delimiter=",",quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

extracting poses and their coords

In [4]:
classname="face"
subclass="smile"
condition="good"

In [30]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False


        results=holistic.process(image)


        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(0,0,0),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1))

        left_hand_poses,right_hand_poses=None,None

        try:
            poses=results.pose_landmarks.landmark
            pose_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in poses]).flatten())

            face_poses=results.face_landmarks.landmark
            face_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face_poses]).flatten())

            #deal with cases where hand isnt detected
            left_hand_poses=results.left_hand_landmarks
            if left_hand_poses:
                left_hand_poses=results.left_hand_landmarks.landmark
                left_hand_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in left_hand_poses]).flatten())
            else:
                left_hand_row=[0]*(21*4)

            right_hand_poses=results.right_hand_landmarks
            if right_hand_poses:
                right_hand_poses=results.right_hand_landmarks.landmark
                right_hand_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in right_hand_poses]).flatten())
            else:
                right_hand_row=[0]*(21*4)

            row=pose_row+left_hand_row+right_hand_row+face_row
            row.insert(0,classname)
            row.insert(1,subclass)
            row.insert(2,condition)

            with open("coordinates.csv",mode="a",newline='') as f:
                csv_writer=csv.writer(f,delimiter=",",quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass


        cv2.imshow("feed",image)
        
        if cv2.waitKey(100) & 0xFF==ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

2175

In [17]:
poses=results.pose_landmarks.landmark
pose_row=list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in poses]).flatten())
pose_row

[0.5374516248703003,
 0.5445611476898193,
 -1.3374149799346924,
 0.998854398727417,
 0.5655264258384705,
 0.4737181067466736,
 -1.2603517770767212,
 0.9980114698410034,
 0.5820627212524414,
 0.47362950444221497,
 -1.2609684467315674,
 0.9981318116188049,
 0.5993592739105225,
 0.4750102460384369,
 -1.2611280679702759,
 0.9976445436477661,
 0.5062063932418823,
 0.4792427718639374,
 -1.2729709148406982,
 0.9982413053512573,
 0.48467928171157837,
 0.48172345757484436,
 -1.272695779800415,
 0.9984803199768066,
 0.4653407037258148,
 0.4837382733821869,
 -1.2733867168426514,
 0.9984688758850098,
 0.6245622634887695,
 0.5127922892570496,
 -0.7620664238929749,
 0.9975228905677795,
 0.4338712692260742,
 0.521263599395752,
 -0.804248034954071,
 0.9988958835601807,
 0.5694273710250854,
 0.6176755428314209,
 -1.146939754486084,
 0.9988582730293274,
 0.4985264241695404,
 0.6204915642738342,
 -1.1599647998809814,
 0.9993542432785034,
 0.7693522572517395,
 0.8996641635894775,
 -0.45520442724227905,
 0